This Notebook is used to preprocess the training data. It takes video as the input and returns a sampled video

In [1]:
from google.colab import drive
drive.flush_and_unmount
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [2]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 16.1MB/s 


In [3]:
import time
import os

import cv2
import random

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torchvision.transforms as transforms

import math
from tqdm import tqdm

from random import sample
import numpy as np
from collections import Counter
import sys
from PIL import Image


def set_project_folder_dir(if_open_new_folder, local_dir, use_model_folder_dir=False, mode=None):
    if use_model_folder_dir:
        folder_dir = os.path.join(os.path.normpath(local_dir + os.sep + os.pardir), mode)
        create_folder_dir_if_needed(folder_dir)
    else:
        if if_open_new_folder != 'False':
            folder_dir = open_new_folder(if_open_new_folder, local_dir)
        else:
            folder_dir = local_dir
    return folder_dir


def open_new_folder(if_open_new_folder, local_dir):
    if if_open_new_folder == 'True':
        folder_name = time.strftime("%Y%m%d-%H%M%S")
    else:
        folder_name = 'debug'
    folder_dir = os.path.join(local_dir, folder_name)
    create_folder_dir_if_needed(folder_dir)
    return folder_dir


def save_setting_info(args, device, folder_dir):
    setting_file_name = os.path.join(folder_dir, 'setting_info.txt')
    args_dict = args.__dict__
    with open(setting_file_name, 'w') as f:
        for key, value in args_dict.items():
            f.write(key + ' : ' + str(value) + '\n')
        f.write(str(device))




def get_data(mode, video_names, list, number_of_classes, labels=[]):
    # setting the data files as a list so the not overpower the system
    for video_name in video_names:
        if mode == 'train':
            video_name, label = video_name.split(' ')
            label = int(label.rstrip('\n'))
            if number_of_classes is None or label in range(1, number_of_classes + 1):
                labels.append(label - 1)
                list.append(video_name.split('.')[0])
            else:
                continue
        else:
            list.append(video_name.split('.')[0])
    return list, labels


def get_video_list(ucf_list_root, number_of_classes, folder_dir):
    # ====== get a list of video names ======
    video_names_train, video_names_test, labels = [], [], []
    sample_train_test_split = str(sample(range(1, 4), 1)[0])
    with open(os.path.join(folder_dir, 'setting_info.txt'), 'a+') as f:
        f.write('\nThe test/train split that we have train on is {}'.format(sample_train_test_split))
    for file_name in os.listdir(ucf_list_root):
        file_path = os.path.join(ucf_list_root, file_name)
        if 'train' in file_name and sample_train_test_split in file_name:
            with open(file_path) as f:
                video_names = f.readlines()
            video_names_train, labels = get_data('train', video_names, video_names_train, number_of_classes, labels)
        elif 'classInd' in file_name:
            with open(file_path) as f:
                labels_decoder = f.readlines()
            labels_decoder_dict = {int(x.split(' ')[0]) - 1: x.split(' ')[1].rstrip('\n') for x in labels_decoder}
        elif 'test' in file_name and sample_train_test_split in file_name:
            with open(file_path) as f:
                video_names = f.readlines()
            video_names_test, _ = get_data('test', video_names, video_names_test, number_of_classes)

    return video_names_train, video_names_test, labels, labels_decoder_dict




def create_folder_dir_if_needed(folder_save_dir):
    if not os.path.exists(folder_save_dir):
        os.makedirs(folder_save_dir)





def set_transforms(mode):
    if mode == 'train':
        transform = transforms.Compose(
            [transforms.Resize(256),  # this is set only because we are using Imagenet pre-train model.
             transforms.RandomCrop(224),
             transforms.RandomHorizontalFlip(),
             transforms.ToTensor(),
             transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                  std=(0.229, 0.224, 0.225))
             ])
    elif mode == 'test' or mode == 'val':
        transform = transforms.Compose([transforms.Resize((224, 224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                                             std=(0.229, 0.224, 0.225))])
    return transform


def create_new_video(save_path, video_name, image_array):
    
    (h, w) = image_array[0].shape[:2]
    if len(video_name.split('/')) > 1:
        video_name = video_name.split('/')[1]
    else:
        video_name = video_name.split('.mp4')[0]
        video_name = video_name + '.avi'
    save_video_path = os.path.join(save_path, video_name)
    output_video = cv2.VideoWriter(save_video_path, cv2.VideoWriter_fourcc(*'MJPG'), 5, (w, h), True)
    for frame in range(len(image_array)):
        output_video.write(image_array[frame])
    output_video.release()
    cv2.destroyAllWindows()


#########
def setting_sample_rate(num_frames_to_extract, sampling_rate, video, fps, ucf101_fps):
    video.set(cv2.CAP_PROP_POS_AVI_RATIO, 1)
    
    # video_length = video.get(cv2.CAP_PROP_POS_MSEC)
    # print("VideoLenght", video_length)

    cal_fps = video.get(cv2.CAP_PROP_FPS)

    print("FPS :",cal_fps)
    
    
    num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    # print( num_frames )

    # num_frames = int(video_length * fps)
    # print("number of frames", num_frames)

    if num_frames_to_extract == 'all':
        sample_start_point = 0
        if fps != ucf101_fps and sampling_rate != 0:
            sampling_rate = math.ceil(fps / (ucf101_fps / sampling_rate))
    elif num_frames < (num_frames_to_extract * sampling_rate):
        sample_start_point = 5
        sampling_rate = 3
    else:
        # p = num_frames - 151

        #25
        q = random.randint(0,num_frames//15)
        sample_start_point = q
        if cal_fps > 30:
          sampling_rate = 20
        #sample(range(num_frames - (num_frames_to_extract * sampling_rate)), 1)[0]
    return sample_start_point, sampling_rate, num_frames

########
def capture_and_sample_video(row_data_dir, video_name, num_frames_to_extract, sampling_rate, fps, save_path,
                             ucf101_fps, processing_mode):
    video = cv2.VideoCapture(os.path.join(row_data_dir, video_name))
    # if fps == 'Not known':

    fps = video.get(cv2.CAP_PROP_FPS)
    ucf101_fps = video.get(cv2.CAP_PROP_FPS)

    video_width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    video_height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    print("video ", video_name)

    sample_start_point, sampling_rate, num_frames = setting_sample_rate(num_frames_to_extract, sampling_rate, video,
                                                                        fps, ucf101_fps)
    # ====== setting the video to start reading from the frame we want ======
    
    print("details: ",sample_start_point, sampling_rate, num_frames )
   

    image_array = []
    if num_frames_to_extract == 'all':
        num_frames_to_extract = int(num_frames / sampling_rate) if sampling_rate != 0 else num_frames
    # print("Nmber of frames to extract ", num_frames_to_extract)
    
    if processing_mode == 'live':
        transform = set_transforms(mode='test')
    for frame in range(num_frames_to_extract):
        video.set(1, sample_start_point)
        success, image = video.read()
        if not success:
            print('')#Error in reading frames from row video')
        else:
            # image = image[l:r, t:b] #top-botton, left-right
            RGB_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if processing_mode == 'live' else image
            image = Image.fromarray(RGB_img.astype('uint8'), 'RGB')
            if processing_mode == 'live':
                image_array += [transform(image)]
            else:
                image_array += [np.uint8(image)]
        sample_start_point = sample_start_point + sampling_rate
    video.release()
    if processing_mode == 'main' and success:
        create_new_video(save_path, video_name, image_array)

    else:
         print('Error in reading frames from row video')
    return image_array, [video_width, video_height]




You can change the preprocesing according to your requirements.
sampling rate and number of frames to exract are the important arguments which will change the model behaviour. Rename the output folder for every class.



In [4]:
import glob
import argparse
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook #used when I run in colab/GCloud
import os


parser = argparse.ArgumentParser(description='UCF101 Action Recognition preprocessing data, LRCN architecture')

#path of the dataset
parser.add_argument('--row_data_dir', default=r'/content/drive/My Drive/directed studies/talk', type=str,
                    help='path to find the UCF101 row data')
                     type=str, help='path to find the UCF101 list splitting the data to train and test')

parser.add_argument('--sampling_rate', default=5, type=int, help='how to sample the data')
parser.add_argument('--ucf101_fps', default=60, type=int, help='FPS of the UCF101 dataset')
parser.add_argument('--num_frames_to_extract', default=15, type=int, help='The number of frames what would be extracted from each video')
parser.add_argument('--video_file_name', default='y2mate.com - cute_happy_baby_crawling_BkJ6FJ2jJEQ_360p.mp4', type=str,
                    help='the video file name we would process, if none the script would run on all of the video files in the folder')
parser.add_argument('--dataset', default='UCF101', type=str,
                    help='the dataset name. options = youtube, UCF101')


def main_procesing_data(args, folder_dir, sampled_video_file=None, processing_mode='main'):
    """"
       Create the sampled data video,
       input - video, full length.
       function - 1. Read the video using CV2
                  2. from each video X (args.sampling_rate) frames are sampled reducing the FPS by args.sampling_rate (for example from 25 to 2.5 FPS)
                  3. The function randomly set the start point where the new sampled video would be read from, and Y(args.num_frames_to_extract) continues frames are extracted.
                  4. if processing_mode == 'main' The Y continues frames are extracted and save to a new video if not the data in tensor tyoe mode is passed to the next function
       Output: videos in length of X frames
       """
    if args.dataset == 'UCF101':
        videopath = glob.glob(args.row_data_dir+"/*")

        video_list = []
        for path in videopath:
          video_list.append(path)
        
        with tqdm(total=len(video_list)) as pbar:
        
            for video_name in video_list:
                print(video_name)
                video_name = video_name.split('/')[-1].rstrip('\n')
                capture_and_sample_video(args.row_data_dir, video_name, args.num_frames_to_extract,
                                          args.sampling_rate, args.ucf101_fps, folder_dir,
                                          args.ucf101_fps, processing_mode)
                pbar.update(1)
           


args = parser.parse_args("")
global_dir = "/content/drive/My Drive/directed studies/"

#The folder name where the output will be stored
folder_name = 'testing{}_sampled_data_video_sampling_rate_{}_num frames extracted_{}'.format(args.dataset, args.sampling_rate, args.num_frames_to_extract)

folder_dir = os.path.join(global_dir, folder_name)
create_folder_dir_if_needed(folder_dir)
save_setting_info(args, "cpu", folder_dir)
main_procesing_data(args, folder_dir)

  0%|          | 0/120 [00:00<?, ?it/s]

/content/drive/My Drive/directed studies/talk/American_History_X_talk_h_nm_np1_fr_goo_20.avi
video  American_History_X_talk_h_nm_np1_fr_goo_20.avi
FPS : 30.0
details:  8 5 231


  1%|          | 1/120 [00:01<02:33,  1.29s/it]

/content/drive/My Drive/directed studies/talk/American_History_X_talk_u_nm_np1_fr_goo_21.avi
video  American_History_X_talk_u_nm_np1_fr_goo_21.avi
FPS : 30.0
details:  2 5 117


  2%|▏         | 2/120 [00:02<02:14,  1.14s/it]

...

/content/drive/My Drive/directed studies/talk/Fellowship_2_talk_h_nm_np1_fr_bad_6.avi
video  Fellowship_2_talk_h_nm_np1_fr_bad_6.avi
FPS : 30.0
details:  6 5 168


  8%|▊         | 9/120 [00:08<01:35,  1.16it/s]

/content/drive/My Drive/directed studies/talk/Fellowship_3_talk_h_nm_np1_fr_goo_1.avi


  8%|▊         | 9/120 [00:08<01:43,  1.08it/s]


KeyboardInterrupt: ignored

After preprocessing, we get seperate folders. Copy all the smoking class videos into 1 folder and non smoking class video into another folder.

Rename the videos using the code below

In [6]:
#For smoking class

import os 
  
for count, filename in enumerate(os.listdir('/content/drive/My Drive/directed studies/Smoking_class_video_folder/')): 
    if ".avi" in filename:
      dst ="1-Smoking" + str(count) + ".avi"
      src ='/content/drive/My Drive/directed studies/Smoking_class_video_folder/'+ filename 
      dst ='/content/drive/My Drive/directed studies/Smoking_class_video_folder/'+ dst
        
      # rename() function will 
      # rename all the files 
      os.rename(src, dst) 

In [ ]:
#For Non-smoking class

import os 
  
for count, filename in enumerate(os.listdir('/content/drive/My Drive/directed studies/Non-Smoking_class_video_folder/')): 
    if ".avi" in filename:
      dst ="1-Smoking" + str(count) + ".avi"
      src ='/content/drive/My Drive/directed studies/Non-Smoking_class_video_folder/'+ filename 
      dst ='/content/drive/My Drive/directed studies/Non-Smoking_class_video_folder/'+ dst
        
      # rename() function will 
      # rename all the files 
      os.rename(src, dst) 

Finally, Combine the videos of both classes to create the final dataset folder